In [1]:
import pandas as pd

In [2]:
df1= pd.DataFrame(pd.read_csv("data/gemstone.csv"))

In [3]:
df1.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df1.drop(columns=['id'],inplace=True)

In [5]:
x = df1.drop(columns=['price'],axis=1)
y = df1["price"]

In [6]:
numerical_columns = x.select_dtypes(exclude= 'object').columns
categorical_columns = x.select_dtypes(include= 'object').columns

In [7]:
pd.DataFrame(categorical_columns)

,0
0,cut
1,color
2,clarity


In [8]:
df1["clarity"].unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [9]:
cut_categories=['Fair','Good', 'Very Good','Premium', 'Ideal']
color_categories = ['D','E','F','G','H','I','J'] #opp
calirty_categories = ['I1','SI2','SI1','VS2','VS1','VVS2', 'VVS1','IF' ]

In [10]:
from sklearn.impute import SimpleImputer #missing values
from sklearn.preprocessing import StandardScaler #Feature Scaling(numerical datatype)
from sklearn.preprocessing import OrdinalEncoder # to rank the categorical variables
#pipeline
from sklearn.pipeline import Pipeline # to club everything together
from sklearn.compose import ColumnTransformer # Begin the work


In [11]:
num_pipeline = Pipeline(
    steps= [
        ("imputer", SimpleImputer(strategy="median")), #even if no missing values in training data but in future there can be
        ("scaler", StandardScaler())

    ]
)

cat_pipeline = Pipeline(
    steps= [
        ("imputer", SimpleImputer(strategy="most_frequent")), #even if no missing values in training data but in future there can be
        ("OrdinalEncoder", OrdinalEncoder(categories=[cut_categories,color_categories,calirty_categories])),
        ("scaler", StandardScaler()) #can keep it or not depends on you
    ]
)

# change the rows mentioned in numerical_colums with the instuction in num_pipline
preprocessor = ColumnTransformer(
    [
        ('num_pipeline',num_pipeline, numerical_columns),
        ('cat_pipeline',cat_pipeline, categorical_columns)
    ]
)


train test split

In [25]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=30)

In [28]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [29]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.473767,3.220176,-0.641637,0.445161,0.398319,0.747248,-3.151612,-0.318068,0.019657
1,-0.845697,0.075806,-0.120391,-0.880410,-0.854100,-0.862659,0.873627,0.297795,-0.646663
2,0.733333,-0.201638,0.922103,0.823895,0.897472,0.848774,-1.138993,0.913659,-0.646663
3,-0.196781,0.260769,-0.641637,-0.050801,-0.010079,-0.006943,0.873627,1.529522,0.019657
4,-0.542870,-0.571564,0.400856,-0.465605,-0.436627,-0.485564,0.873627,0.913659,0.685978


In [15]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.564500,-0.941490,-0.641637,-0.429535,-0.463854,-0.500067,-0.132683,-0.933932,0.019657
1,-0.175150,1.000621,-0.120391,-0.041783,-0.028230,0.036568,-1.138993,0.913659,-0.646663
2,-1.062003,0.260769,-0.120391,-1.304232,-1.298800,-1.268762,0.873627,0.913659,2.684938
3,0.971269,-0.201638,1.964596,1.049332,0.997302,0.979307,-0.132683,0.297795,0.019657
4,-0.932220,-1.311416,0.400856,-1.006654,-0.990233,-1.065710,-0.132683,-0.933932,0.685978


In [16]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [17]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [18]:
regression.coef_

array([ 6434.78387743,  -132.64285991,   -70.72462747, -1744.76501257,
        -471.40887345,   -70.2527634 ,    71.08211437,  -462.11164254,
         651.0562136 ])

In [19]:
regression.intercept_

3968.3548309328453

In [20]:
import numpy as np
def model_evaluate(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae,mse, rmse, r2_square

## Train multiple models

In [21]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae,mse, rmse, r2_square = model_evaluate(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("MAE:",mae)
    print("MSE:",mse)
    print("RMSE:",rmse)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
MAE: 677.565219031756
MSE: 1042258.6481907823
RMSE: 1020.9106955021983
R2 score 93.63482324438802


Lasso
Model Training Performance
MAE: 678.6412565975306
MSE: 1042192.5916603323
RMSE: 1020.8783432223119
R2 score 93.63522665815954


Ridge
Model Training Performance
MAE: 677.5922524952755
MSE: 1042258.1722427476
RMSE: 1020.910462402432
R2 score 93.63482615104978


